# 2d quadcopter simulation
This is intended to be an exercise in understanding the logic of a basic control system by creating one essentially from scratch. Of course, to test this, one needs to have an environment, or a robot to test on, which is an opportunity for me to work further on simulation skills. Admittedly, something else might be easier, but quadcopters capture my interest.


In [1]:
import math
import matplotlib.pyplot as plt

In [2]:
def AddVectors(*x):
    """
    """
    out = [0]*len(x[0])
    for i in x:
        k = 0
        #print("i = {}\n".format(i))
        for j in i:
            #print("j = {}\n".format(j))
            out[k] += j
            k += 1
    return out

I think about these problems very much from an object oriented perspective. By creating code versions of physical components, I can make the code easier to read and understand.

The propeller has mass and thrust properties, which could be tweaked for more realism or testing out parts in a more realistic scenario.

In [3]:
class Propeller():
    """
    """
    def __init__(self):
        self.mass = 1
        self.thrust_range = (0,1000)
        self.thrust = 0

    def Thrust(self, speed_change):
        """
        account for propeller limits
        """
        self.thrust += min(speed_change, self.thrust_range[1])
        self.thrust = max(self.thrust, self.thrust_range[0])

In [4]:

class Quadcopter():
    """
    """
    def __init__(self):
        """
        """
        self.prop1 = Propeller()
        self.prop2 = Propeller()

        self.arm1_coord_local = (2,0)
        self.arm2_coord_local = (-2,0)

        self.body_mass = 3
        self.mass = self.body_mass + self.prop1.mass + self.prop2.mass

        self.pos = [0,0]
        self.vel = [0,0]
        self.acc = [0,0]
        self.angle   = 0
        self.ang_vel = 0
        self.ang_acc = 0

        self.Kp = 1
        self.Kv = 1
        self.Ka = 1
        self.Kw = 1
        
        
    def Control(self, target_pos, target_vel):
        """
        figure out how to control propellers to get to objective
        """
        g = [0,-1]
        
        # gives force in x/y, not left/right rotors
        #F_desired = AddVectors([self.Kp*target_pos, -self.Kp*self.pos], [self.Kv*target_vel, -self.Kv*self.vel], g)
        pos_adjust = [self.Kp*(t-p) for t,p in zip(target_pos, self.pos)]
        vel_adjust = [self.Kv*(t-v) for t,v in zip(target_pos, self.vel)]
        F_desired  = [p + v + j for p, v, j in zip(pos_adjust, vel_adjust, g)]
        Thrust_desired = math.sqrt(F_desired[0]**2 + F_desired[1]**2)
        
        # direction of approach
        Angle_desired = math.atan2(F_desired[1], F_desired[0]) + math.atan2(target_vel[1], target_vel[0])
        
        # copter needs to tilt to move laterally
        T_desired = self.Ka*(Angle_desired-self.angle) - self.Kw*(self.ang_vel)

        # set propeller based on needed acceleration and needed torque
        self.prop1.Thrust(Thrust_desired/2 + T_desired/2)
        self.prop2.Thrust(Thrust_desired/2 - T_desired/2)
        
    # External Forces
    # Apply a linear force to the copter
    def ApplyForce(self, force, *force_args, **force_kwargs):
        """
        """
        if not force: return
        f = force
        if force_args or force_kwargs:
            f = force(*force_args, **force_kwargs)
        f[:] = [i / self.mass for i in f]
        self.acc = AddVectors(self.acc, f)
        
    # Apply a torque to the copter
    def ApplyTorque(self, torque, *torque_args, **torque_kwargs):
        """
        """
        if not torque: return
        t = torque
        if torque_args or torque_kwargs:
            t =  torque(*torque_args, **torque_kwargs)
        
        self.ang_acc += t / (self.mass*self.arm1_coord_local[0]**2)
    

In [6]:
Q = Quadcopter()

# Translating forces into velocities and positions
def UpdatePos(Q):
    """
    get position change from acceleration
    """
    #Q.vel = AddVectors(Q.vel, Q.acc)
    Q.vel[0] += Q.acc[0]
    Q.vel[1] += Q.acc[1]
    
    #Q.pos = AddVectors(Q.pos, Q.vel)
    Q.pos[0] += Q.vel[0]
    Q.pos[1] += Q.vel[1]

def UpdateAng(Q):
    """
    get angle change from angular acceleration
    """
    Q.ang_vel = Q.ang_vel + Q.ang_acc
    Q.angle = Q.angle + Q.ang_vel

# Simulation: propeller -> movement
def CalcLinAcc(Q, ext_force = None): # doesn't account for external forces
    """
    get needed linear acceleration
    """
    # calculates the linear acceleration from propellers, needs to be vector
    acc_mag = (Q.prop1.thrust + Q.prop2.thrust)/(2.0 * Q.mass)
    #print(Q.prop1.thrust, Q.prop2.thrust)
    Q.acc = [acc_mag*math.sin(Q.angle), acc_mag*math.cos(Q.angle)]
    Q.ApplyForce(ext_force)

def CalcAngAcc(Q, ext_torque = None):
    """
    get needed angular acceleration
    """
    # calculates the angular acceleration from propellers
    Q.ang_acc = Q.prop1.thrust/(Q.prop1.mass*Q.arm1_coord_local[0]**2) - Q.prop2.thrust/(Q.prop2.mass*Q.arm2_coord_local[0]**2)
    Q.ApplyTorque(ext_torque)

def Display(Q):
    """
    """
    pass

t = 0
runtime = 7000

running = True
while running:
    UpdatePos(Q)
    UpdateAng(Q)
    CalcLinAcc(Q)
    CalcAngAcc(Q)

    #print(f"position is {Q.pos}")
    #print(f"velocity is {Q.vel}")
    #print(f"acceleration is {Q.acc}")

    Q.Control(target_pos=[10,10], target_vel=[5,5])

    # Display()
    #plt.figure()
    #plt.plot(Q.pos[0], Q.pos[1])
    #plt.show()

    if t > runtime: running = False
    t += 1